# Langgraph memory agent

In [ ]:
%pip install langchain-community langchain-core

In [ ]:
%pip install -U langgraph

In [ ]:
%pip install pinecone

In [ ]:
%pip install -U langmem

In [ ]:
%pip install openai

In [ ]:
%pip install dotenv

In [56]:
from langmem import create_memory_manager

In [59]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())
from openai import OpenAI

In [60]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [61]:
from pydantic import BaseModel, Field

class Episode(BaseModel):
    """Record the agent's perspective of an interaction, capturing key internal thought processes to facilitate learning over time."""
    observation: str = Field(..., description="The context and setup - what happened")
    thoughts: str = Field(
        ...,
        description="Internal reasoning process and observations of the agent in the episode that led to the correct action and result. 'I ...'",
    )
    action: str = Field(
        ...,
        description="What was done, how, and in what format. (Include whatever is salient to the success of the action). 'I ...'",
    )
    result: str = Field(
        ...,
        description="Outcome and retrospective. What did you do well? What could you do better next time? 'I ...'",
    )

In [62]:
manager = create_memory_manager(
    "gpt-4o-mini",
    schemas=[Episode],
    instructions="Extract examples of successful explanations, capturing the full chain of reasoning. Be concise in your explanations and precise in the logic of your reasoning.",
    enable_inserts=True,
)

In [63]:
conversation = [
    {"role": "user", "content": "What's a binary tree? I work with family trees if that helps"},
    {"role": "assistant", "content": "A binary tree is like a family tree, but each parent has at most 2 children. Here's a simple example:\n   Bob\n  /  \\\n Amy  Carl\n\n Just like in family trees, we call Bob the 'parent' and Amy and Carl the 'children'."},
    {"role": "user", "content": "Oh, that makes sense! So in a binary search tree, would it be like organizing a family by age?"},
]

episodes = manager.invoke({"messages": conversation})
print(episodes[0])


ExtractedMemory(id='575e91cb-6926-4ddf-9bac-0fcd13d7ef63', content=Episode(observation='The human asked for a definition of a binary tree, relating it to their experience with family trees.', thoughts='I need to explain the concept of a binary tree in a way that connects with their knowledge of family trees. Using a familiar analogy will help them understand the structure more effectively.', action='I compared a binary tree to a family tree by explaining that each parent can have at most two children, providing a simple diagram with Bob as the parent and Amy and Carl as children.', result='The human understood the analogy and found it relatable, confirming their comprehension.'))


Let's go ahead and try to store the memory in a vector database like Pinecone

In [71]:
from pinecone import Pinecone

In [72]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [73]:
from pinecone import ServerlessSpec

In [74]:
index_name = 'memory-db'

In [18]:
if not pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [75]:
index = pc.Index(index_name)

In [69]:
from pydantic import BaseModel, Field

class Episode(BaseModel):
    observation: str = Field(..., description="Context of the interaction")
    thoughts: str = Field(..., description="Assistant's reasoning during the interaction")
    action: str = Field(..., description="Action taken by the assistant")
    result: str = Field(..., description="Outcome of the action")

In [24]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(model_name='gpt-4o-mini', openai_api_key='OPENAI_API_KEY')

# Initialize conversational memory
memory = ConversationBufferMemory(llm=llm)

# Create the conversation chain
conversation = ConversationChain(llm=llm, memory=memory)

/var/folders/jv/nvnkgz9s2nnbd6d3c005qd5m0000gn/T/ipykernel_89216/3892135470.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-4o-mini', openai_api_key='OPENAI_API_KEY')
/var/folders/jv/nvnkgz9s2nnbd6d3c005qd5m0000gn/T/ipykernel_89216/3892135470.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(llm=llm)
/var/folders/jv/nvnkgz9s2nnbd6d3c005qd5m0000gn/T/ipykernel_89216/3892135470.py:12: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistor

In [35]:

# Define the conversation history
conversation_history = [
    {"role": "user", "content": "What's a binary tree? I work with family trees if that helps."},
    {"role": "assistant", "content": "A binary tree is like a family tree, but each parent has at most two children. Here's a simple example:\n   Bob\n  /  \\\n Amy  Carl\n\nJust like in family trees, we call Bob the 'parent' and Amy and Carl the 'children'."},
    {"role": "user", "content": "Oh, that makes sense! So in a binary search tree, would it be like organizing a family by age?"},
]

# Generate an embedding for the conversation
response = client.embeddings.create(
    input=str(conversation_history),
    model="text-embedding-3-small"
)

embedding = response.data[0].embedding

# Define the episodic memory
episode = Episode(
    observation="The user asked for an explanation of a binary tree and related it to family trees.",
    thoughts="I realized the user was familiar with family trees, which allowed me to relate the concept of a binary tree to something they understood.",
    action="I explained that a binary tree consists of a parent with at most two children and illustrated it with a simple example using names.",
    result="The user expressed understanding, indicating that my explanation was clear and effective."
)

# Upsert the episodic memory into Pinecone
index.upsert(vectors=[("unique-id-1", embedding, episode.model_dump())])


{'upserted_count': 1}

In [36]:
user_query = "Can you explain binary trees in the context of family relationships?"

In [37]:
response = client.embeddings.create(
    input=user_query,
    model="text-embedding-3-small"
)
query_embedding = response.data[0].embedding

In [40]:
len(query_embedding)

1536

In [41]:
search_results = index.query(vector=query_embedding, top_k=5, include_metadata=True)

In [42]:
retrieved_memories = [match['metadata'] for match in search_results['matches']]

In [43]:
retrieved_memories

[{'action': 'I explained that a binary tree consists of a parent with at most two children and illustrated it with a simple example using names.',
  'observation': 'The user asked for an explanation of a binary tree and related it to family trees.',
  'result': 'The user expressed understanding, indicating that my explanation was clear and effective.',
  'thoughts': 'I realized the user was familiar with family trees, which allowed me to relate the concept of a binary tree to something they understood.'}]

In [54]:
# Step 1: Format Retrieved Memories
context = "\n\n".join([
    f"Observation: {memory['observation']}\nThoughts: {memory['thoughts']}\nAction: {memory['action']}\nResult: {memory['result']}"
    for memory in retrieved_memories
])

# Step 2: Construct messages for chat-based model
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Use past memories if they are relevant."},
        {"role": "system", "content": f"Here are some relevant past memories:\n{context}"},
        {"role": "user", "content": user_query}
    ],
    temperature=0,
    max_tokens=1000
)

# Step 3: Extract the reply
assistant_reply = response.choices[0].message.content.strip()


In [55]:
assistant_reply

'Sure! A binary tree can be a great way to visualize family relationships, especially when considering how each person can have a limited number of direct descendants.\n\nIn a binary tree:\n\n- Each node represents a person (like a family member).\n- The top node is the "root" of the tree, which could represent the oldest generation, such as a grandparent.\n- Each person (node) can have at most two children (nodes), which could represent their direct descendants, like a parent having two children.\n\nFor example, let\'s say we have a grandparent named "Grandma." She has two children: "Aunt" and "Uncle." In this case, Grandma is the root of the tree, Aunt and Uncle are her children (the left and right children of the root node).\n\nIf Aunt has two children, "Cousin1" and "Cousin2," they would be the left and right children of Aunt. Uncle, on the other hand, might have one child, "Cousin3," who would be the left child of Uncle, while Uncle has no right child.\n\nSo, the structure would l

Utilizing langgraph

In [ ]:
%pip install langchain-pinecone

In [89]:
from langchain_pinecone import PineconeVectorStore

In [85]:
from langchain.embeddings.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [90]:
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [91]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

In [92]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

/var/folders/jv/nvnkgz9s2nnbd6d3c005qd5m0000gn/T/ipykernel_89216/988334424.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [98]:
# User input and assistant output
user_input = "Hello my name is Jonas"
assistant_output = "Hi Jonas, nice to meet you! How can I assist you today?"

# Save the context
memory.save_context({"input": user_input}, {"output": assistant_output})


In [99]:
# Load the current summary
summary = memory.load_memory_variables({})
print(summary['history'])


The human asks the AI to explain binary trees. The AI explains that a binary tree is a data structure with nodes that can have up to two children. The human then asks how this relates to family trees, and the AI explains that while both represent hierarchical relationships between nodes, binary trees are based on the order of insertion rather than familial relationships. The human introduces themselves as Jonas and the AI greets them and offers assistance, introducing itself as an AI.


In [100]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=llm, memory=memory)

# Engage in a conversation
response = conversation.predict(input="What is my name?")
print(response)

 Your name is Jonas. Is there anything else you would like to know, Jonas? I am here to assist you.


In [101]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embedding_model, text_key="summary")

In [102]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

In [103]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)


In [104]:
# Example conversation inputs and outputs
user_input = "Hello, can you explain binary trees?"
assistant_output = "Certainly! A binary tree is a data structure where each node has at most two children."

# Save the context to update the summary
memory.save_context({"input": user_input}, {"output": assistant_output})

# Retrieve the updated summary
summary = memory.load_memory_variables({})["history"]

# Generate embedding for the summary
summary_embedding = embedding_model.embed_query(summary)

# Upsert the summary and its embedding into Pinecone
vector_store.add_texts(texts=[summary], embeddings=[summary_embedding])


['0e9f74a2-5834-4319-b056-a3d13e186647']